In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,RobustScaler,MinMaxScaler, LabelEncoder, StandardScaler
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, roc_auc_score, plot_roc_curve, f1_score,confusion_matrix , accuracy_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV,cross_validate
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor, ExtraTreesClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBClassifier
import sklearn.metrics as met
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

import pickle
import missingno as msno
import lazypredict
from lazypredict.Supervised import LazyClassifier, LazyRegressor

# <center> Intro <center>
1. Problem = how to predict the company will survive or not with company current condition, and we position ourselves as company consultant, and we must give our client insight to keep their company running well
2. Data =
>* what is being predicted? = Company bankruptcy paramater
>* what is needed in prediction? = selecting the feature we have that impact the bankrupt for company, so we can reduce the risk of the company going bankrupt, and can increase the company's chances of continuing to work
3. Machine Learning Objective = Maximize chance to survive
4. Action = dont let the predict fail, we say the company will survive but eventually go bankrupt, and the company will go bankrupt while surviving
5. Value = keep the company from going bankrupt

In [5]:
df = pd.read_csv("data.csv")
df

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.37,0.42,0.41,0.60,0.60,1.00,0.80,0.81,0.30,...,0.72,0.01,0.62,0.60,0.83,0.29,0.03,0.56,1,0.02
1,1,0.46,0.54,0.52,0.61,0.61,1.00,0.80,0.81,0.30,...,0.80,0.01,0.62,0.61,0.84,0.28,0.26,0.57,1,0.02
2,1,0.43,0.50,0.47,0.60,0.60,1.00,0.80,0.81,0.30,...,0.77,0.04,0.62,0.60,0.84,0.29,0.03,0.56,1,0.02
3,1,0.40,0.45,0.46,0.58,0.58,1.00,0.80,0.81,0.30,...,0.74,0.00,0.62,0.58,0.83,0.28,0.03,0.56,1,0.02
4,1,0.47,0.54,0.52,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.60,0.84,0.28,0.02,0.58,1,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.49,0.54,0.54,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.60,0.84,0.28,0.03,0.57,1,0.03
6815,0,0.48,0.54,0.52,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.60,0.84,0.28,0.03,0.57,1,0.04
6816,0,0.47,0.53,0.52,0.61,0.61,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.61,0.84,0.28,0.03,0.57,1,0.10
6817,0,0.51,0.56,0.55,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.00,0.62,0.61,0.84,0.28,0.03,0.57,1,0.04


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-null   float64
 8    After-tax net Int

the dataset contain 2 data types : int and float

In [4]:
df.describe()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,...,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00,6819.00
mean,0.03,0.51,0.56,0.55,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,18629417.81,0.62,0.61,0.84,0.28,0.03,0.57,1.00,0.05
std,0.18,0.06,0.07,0.06,0.02,0.02,0.01,0.01,0.01,0.01,...,0.04,376450059.75,0.01,0.02,0.01,0.01,0.02,0.01,0.00,0.05
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
25%,0.00,0.48,0.54,0.53,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.60,0.84,0.28,0.03,0.57,1.00,0.02
50%,0.00,0.50,0.56,0.55,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.00,0.62,0.61,0.84,0.28,0.03,0.57,1.00,0.03
75%,0.00,0.54,0.59,0.58,0.61,0.61,1.00,0.80,0.81,0.30,...,0.83,0.01,0.62,0.61,0.84,0.28,0.03,0.57,1.00,0.05
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,9820000000.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


# <center> Feature Selection 

In [7]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [8]:
X = df.drop(['Bankrupt?'], axis = 1)
y = df['Bankrupt?']

In [9]:
selector = SelectKBest(mutual_info_classif,k=9)
selector.fit(X, y)
X.columns[selector.get_support()]

Index([' ROA(A) before interest and % after tax',
       ' Continuous interest rate (after tax)',
       ' Persistent EPS in the Last Four Seasons',
       ' Per Share Net profit before tax (Yuan ¥)', ' Debt ratio %',
       ' Borrowing dependency', ' Net profit before tax/Paid-in capital',
       ' Net Income to Total Assets', ' Net Income to Stockholder's Equity'],
      dtype='object')

### MUTUAL INFO CLASSIF
Estimate mutual information for a discrete target variable.

Mutual information (MI) [1] between two random variables is a non-negative value, which measures the dependency between the variables. It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency.

The function relies on nonparametric methods based on entropy estimation from k-nearest neighbors distances as described in [2] and [3]. Both methods are based on the idea originally proposed in [4].

It can be used for univariate features selection

make a new dataframe using selected feature

In [10]:
df_model = df[['Bankrupt?',
       ' ROA(A) before interest and % after tax',
       ' Persistent EPS in the Last Four Seasons',
       ' Per Share Net profit before tax (Yuan ¥)', 
       ' Debt ratio %',
       ' Net worth/Assets', 
       ' Borrowing dependency',
       ' Net profit before tax/Paid-in capital', 
       ' Net Income to Total Assets',
       " Net Income to Stockholder's Equity"]]

untuk mempermudah pemanggilan kolom, dikarenakan sebelum dilakukan perubahan ini ada spasi untuk nama kolom, dan perubahan ini bertujuan untuk menghindari kesalahan penulisan

In [11]:
df_model.columns = df_model.columns.str.strip()
df_model.columns = df_model.columns.str.replace(" " ,"_")
print("Nama Kolom setelah dirubah","\n",df_model.columns[:10])

Nama Kolom setelah dirubah 
 Index(['Bankrupt?', 'ROA(A)_before_interest_and_%_after_tax',
       'Persistent_EPS_in_the_Last_Four_Seasons',
       'Per_Share_Net_profit_before_tax_(Yuan_¥)', 'Debt_ratio_%',
       'Net_worth/Assets', 'Borrowing_dependency',
       'Net_profit_before_tax/Paid-in_capital', 'Net_Income_to_Total_Assets',
       'Net_Income_to_Stockholder's_Equity'],
      dtype='object')


In [12]:
df_model

,Bankrupt?,ROA(A)_before_interest_and_%_after_tax,Persistent_EPS_in_the_Last_Four_Seasons,Per_Share_Net_profit_before_tax_(Yuan_¥),Debt_ratio_%,Net_worth/Assets,Borrowing_dependency,Net_profit_before_tax/Paid-in_capital,Net_Income_to_Total_Assets,Net_Income_to_Stockholder's_Equity
0,1,0.42,0.17,0.14,0.21,0.79,0.39,0.14,0.72,0.83
1,1,0.54,0.21,0.17,0.17,0.83,0.38,0.17,0.80,0.84
2,1,0.50,0.18,0.14,0.21,0.79,0.38,0.15,0.77,0.84
3,1,0.45,0.19,0.15,0.15,0.85,0.38,0.15,0.74,0.83
4,1,0.54,0.21,0.17,0.11,0.89,0.38,0.17,0.80,0.84
...,...,...,...,...,...,...,...,...,...,...
6814,0,0.54,0.22,0.17,0.12,0.88,0.37,0.17,0.80,0.84
6815,0,0.54,0.22,0.17,0.10,0.90,0.37,0.17,0.80,0.84
6816,0,0.53,0.21,0.17,0.04,0.96,0.37,0.17,0.80,0.84
6817,0,0.56,0.23,0.19,0.09,0.91,0.37,0.18,0.81,0.84


# <center> Confussion Matrix

* *0 = Survive*
* *1 = Bangkrut*

        - TP: Ada perusahaan yang di prediksi Bangkrut dan kenyataannya Bangkrut
        - TN: Ada perusahaan yang di prediksi Survive dan kenyataannya Survive
        - FP: Ada perusahaan yang di prediksi Bangkrut padahal Survive
        - FN: Ada perusahaan yang di prediksi Survive padahal Bangkrut

Tindakan:
* FP: image perusahaan konsultan tidak akurat, tetapi tidak merugikan perusahaan client
* FN: image perusahaan konsultan tidak akurat, merugikan client dan dapat menimbulkan masalah

- > Yang akan di tekan adalah FN, menggunakan recall

# Method Selection

Sebagai langkah awal untuk memilih metode yang terbaik untuk melakukan modelling, kita gunakan lazy predicton sebagai asumsi awal pemilihan metode, untuk selanjutnya kita crosscheck lagi

In [13]:
X = df_model.drop(['Bankrupt?'], axis = 1)
y = df_model['Bankrupt?']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   stratify = y,
                                                   test_size = 0.3,
                                                   random_state = 2021)

In [15]:
LC = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
model,predictions = LC.fit(X_train, X_test, y_train, y_test)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:05<00:00,  5.50it/s]


In [17]:
predictions.sort_values(by='Accuracy')

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
BernoulliNB,0.78,0.79,0.79,0.85,0.01
NearestCentroid,0.90,0.82,0.82,0.93,0.01
DummyClassifier,0.93,0.51,0.51,0.94,0.01
ExtraTreeClassifier,0.95,0.63,0.63,0.95,0.01
DecisionTreeClassifier,0.95,0.62,0.62,0.95,0.04
GaussianNB,0.96,0.68,0.68,0.96,0.01
PassiveAggressiveClassifier,0.96,0.53,0.53,0.95,0.02
BaggingClassifier,0.96,0.59,0.59,0.96,0.18
LabelPropagation,0.96,0.58,0.58,0.96,0.66


Properties of lazy prediction :
1. As of now, it is only based on Supervised learning algorithms(Regression and Classification)
2. Compatible with python version 3.6 and above.
3. Could be run on Command Line Interface(CLI). 
4. Fast in predicting as all the basic model performances for the dataset is given at once.
5. Has an inbuilt Pipeline to scaling and transform the data and handle missing values and change categorical data to numeric. 
6. Provides evaluation metrics on individual models.
7. Shows the time consumed by each model to build.


is the result from lazy predictor accurate? lets see

In [21]:
clf = ExtraTreesClassifier()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print('the result is :')
print('Accuracy : '+str(met.accuracy_score(y_test,y_pred)))
print('f1 score: '+str(met.f1_score(y_test,y_pred,average='weighted')))

the result is :
Accuracy : 0.969208211143695
recall score: 0.9619351271251165


In [22]:
apa = LogisticRegression()
apa.fit(X_train,y_train)
y_pred=apa.predict(X_test)

print('the result is :')
print('Accuracy : '+str(met.accuracy_score(y_test,y_pred)))
print('f1 score: '+str(met.f1_score(y_test,y_pred,average='weighted')))

the result is :
Accuracy : 0.9696969696969697
f1 score: 0.9580401705837246


In [23]:
adb = RandomForestClassifier()
adb.fit(X_train,y_train)
y_pred=adb.predict(X_test)

print('the result is :')
print('Accuracy : '+str(met.accuracy_score(y_test,y_pred)))
print('f1 score: '+str(met.f1_score(y_test,y_pred,average='weighted')))

the result is :
Accuracy : 0.9682306940371457
recall score: 0.9607267184624217


In [24]:
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)

print('the result is :')
print('Accuracy : '+str(met.accuracy_score(y_test,y_pred)))
print('F1 score: '+str(met.f1_score(y_test,y_pred,average='weighted')))

the result is :
Accuracy : 0.966275659824047
F1 score: 0.9577433102948343


its accurate, we got the same value

'weighted':
Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

But we cant 100% believe the lazy predictor, we must re-check again, and using cross validation to make sure the model work better

# Cross Validation

In [26]:
skfold = StratifiedKFold(n_splits=5, random_state=2021, shuffle=False)

for train_index, test_index in skfold.split(X_train,y_train):
    
    print("Train:", train_index, "Test:", test_index)
    X_train_sm, X_val_sm = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_sm, y_val_sm = y_train.iloc[train_index], y_train.iloc[test_index]



X_train_sm = X_train_sm.values
X_val_sm = X_val_sm.values
y_train_sm = y_train_sm.values
y_val_sm = y_val_sm.values


train_unique_label, train_counts_label = np.unique(y_train_sm, return_counts=True)
test_unique_label, test_counts_label = np.unique(y_val_sm, return_counts=True)
print('-' * 84)

print('Label Distributions: \n')
print(train_counts_label/ len(y_train_sm))
print(test_counts_label/ len(y_val_sm))

Train: [ 953  954  955 ... 4770 4771 4772] Test: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 2

In [28]:
kfold = StratifiedKFold(n_splits=5)

# <center> Extra Trees Classifier

In [29]:
accuracy_lst_xtc = []
precision_lst_xtc = []
recall_lst_xtc = []
f1_lst_xtc = []
auc_lst_xtc = []

xtc = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

gsxtc = GridSearchCV(xtc,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)


for train, val in skfold.split(X_train_sm, y_train_sm):
    pipeline_xtc = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), gsxtc)
    model_xtc = pipeline_xtc.fit(X_train_sm[train], y_train_sm[train])
    best_est_xtc = gsxtc.best_estimator_
    prediction_xtc = best_est_xtc.predict(X_train_sm[val])
    
    accuracy_lst_xtc.append(pipeline_xtc.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_xtc.append(precision_score(y_train_sm[val], prediction_xtc))
    recall_lst_xtc.append(recall_score(y_train_sm[val], prediction_xtc))
    f1_lst_xtc.append(f1_score(y_train_sm[val], prediction_xtc))
    auc_lst_xtc.append(roc_auc_score(y_train_sm[val], prediction_xtc))

print('---' * 45)
print('')
print('Extra Trees Classifier results:')
print('')
print("accuracy: {}".format(np.mean(accuracy_lst_xtc)))
print("precision: {}".format(np.mean(precision_lst_xtc)))
print("recall: {}".format(np.mean(recall_lst_xtc)))
print("f1: {}".format(np.mean(f1_lst_xtc)))
#gsxtc.fit(X_train,y_train)
#xtc_best = gsxtc.best_estimator_
print('Best Estimator = {}',best_est_xtc)
print('')
print('---' * 45)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    6.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   22.9s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:   24.8s finished


Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   22.2s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:   24.1s finished


Fitting 5 folds for each of 54 candidates, totalling 270 fits


...

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   22.6s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:   24.3s finished


Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   22.3s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:   24.2s finished


---------------------------------------------------------------------------------------------------------------------------------------

Extra Trees Classifier results:

accuracy: 0.9389863654765908
precision: 0.2743691390846232
recall: 0.5356666666666667
f1: 0.36239868327535263
Best Estimator = {} ExtraTreesClassifier(max_features=3, min_samples_split=3, n_estimators=300)

---------------------------------------------------------------------------------------------------------------------------------------


In [30]:
# Printing the classification report

label = ['0', '1']
pred_xtc_sm = best_est_xtc.predict(X_val_sm)
print(classification_report(y_val_sm, pred_xtc_sm, target_names=label))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98       923
           1       0.39      0.77      0.52        31

    accuracy                           0.95       954
   macro avg       0.69      0.87      0.75       954
weighted avg       0.97      0.95      0.96       954



In [34]:

CM = pd.DataFrame(confusion_matrix(y_val_sm, pred_xtc_sm), columns = ['Survive','Bankrupt'], index = ['Survive','Bankrupt'])
CM

,Survive,Bankrupt
Survive,886,37
Bankrupt,7,24


# <center> XGBoost

In [35]:
accuracy_lst_xgb = []
precision_lst_xgb = []
recall_lst_xgb = []
f1_lst_xgb = []
auc_lst_xgb = []

xgb=XGBClassifier()

xgb_params={"n_estimators":[67,70,100,120],
        'reg_lambda':[2,1],
        'gamma':[0,0.3,0.2,0.1],
        'eta':[0.06,0.05,0.04],
        "max_depth":[3,5],
        'objective':['binary:logistic']}

gsxgb = GridSearchCV(xgb,param_grid = xgb_params, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)


for train, val in skfold.split(X_train_sm, y_train_sm):
    pipeline_xgb = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), gsxgb) # SMOTE happens during Cross Validation not before..
    model_xgb = pipeline_xgb.fit(X_train_sm[train], y_train_sm[train])
    best_est_xgb = gsxgb.best_estimator_
    prediction_xgb = best_est_xgb.predict(X_train_sm[val])
    
    accuracy_lst_xgb.append(pipeline_xgb.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_xgb.append(precision_score(y_train_sm[val], prediction_xgb))
    recall_lst_xgb.append(recall_score(y_train_sm[val], prediction_xgb))
    f1_lst_xgb.append(f1_score(y_train_sm[val], prediction_xgb))
    auc_lst_xgb.append(roc_auc_score(y_train_sm[val], prediction_xgb))

print('---' * 45)
print('')
print('XGBOOST results:')
print('')
print("accuracy: {}".format(np.mean(accuracy_lst_xgb)))
print("precision: {}".format(np.mean(precision_lst_xgb)))
print("recall: {}".format(np.mean(recall_lst_xgb)))
print("f1: {}".format(np.mean(f1_lst_xgb)))
#gsxgb.fit(X_train,y_train)
#gsxgb_best = gsxgb.best_estimator_
print('Best Estimator = {}',best_est_xgb)
print('')
print('---' * 45)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   25.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   53.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed:  1.9min finished


Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   23.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   54.0s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed:  2.0min finished


Fitting 5 folds for each of 192 candidates, totalling 960 fits


...

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   21.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   50.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed:  1.9min finished


Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   21.8s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   51.8s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed:  1.9min finished


---------------------------------------------------------------------------------------------------------------------------------------

XGBOOST results:

accuracy: 0.8994465906829612
precision: 0.20768760852389773
recall: 0.7483333333333333
f1: 0.3248672882059905
Best Estimator = {} XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.06, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0599999987, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=120, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

-------------------------------------------------------------------------------------------------------------------------

In [36]:
# Printing the classification report

label = ['0', '1']
pred_xgb_sm = best_est_xgb.predict(X_val_sm)
print(classification_report(y_val_sm, pred_xgb_sm, target_names=label))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95       923
           1       0.25      0.90      0.40        31

    accuracy                           0.91       954
   macro avg       0.63      0.91      0.67       954
weighted avg       0.97      0.91      0.93       954



In [37]:
CM = pd.DataFrame(confusion_matrix(y_val_sm, pred_xgb_sm), columns = ['Survive','Bankrupt'], index = ['Survive','Bankrupt'])
CM

,Survive,Bankrupt
Survive,841,82
Bankrupt,3,28


# <center> Logistic Regression

In [38]:
accuracy_lst_reg = []
precision_lst_reg = []
recall_lst_reg = []
f1_lst_reg = []
auc_lst_reg = []

log_reg_sm = LogisticRegression()
#log_reg_params = {}
log_reg_params = {"penalty": ['l2'],
                  'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  'class_weight': ['balanced',None],
                  'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

rand_log_reg = GridSearchCV(log_reg_sm,param_grid = log_reg_params, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)


for train, val in skfold.split(X_train_sm, y_train_sm):
    pipeline_reg = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_log_reg) # SMOTE happens during Cross Validation not before..
    model_reg = pipeline_reg.fit(X_train_sm[train], y_train_sm[train])
    best_est_reg = rand_log_reg.best_estimator_
    prediction_reg = best_est_reg.predict(X_train_sm[val])
    
    accuracy_lst_reg.append(pipeline_reg.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_reg.append(precision_score(y_train_sm[val], prediction_reg))
    recall_lst_reg.append(recall_score(y_train_sm[val], prediction_reg))
    f1_lst_reg.append(f1_score(y_train_sm[val], prediction_reg))
    auc_lst_reg.append(roc_auc_score(y_train_sm[val], prediction_reg))


print('---' * 45)
print('')
print('Logistic Regression (SMOTE) results:')
print('')
print("accuracy: {}".format(np.mean(accuracy_lst_reg)))
print("precision: {}".format(np.mean(precision_lst_reg)))
print("recall: {}".format(np.mean(recall_lst_reg)))
print("f1: {}".format(np.mean(f1_lst_reg)))
print('Best Estimator = {}',best_est_reg)
print('')
print('---' * 45)


Fitting 5 folds for each of 70 candidates, totalling 350 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 265 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 350 out of 350 | elapsed:    2.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 70 candidates, totalling 350 fits


[Parallel(n_jobs=4)]: Done 265 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 350 out of 350 | elapsed:    2.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 70 candidates, totalling 350 fits


...

[Parallel(n_jobs=4)]: Done 265 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 350 out of 350 | elapsed:    2.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 70 candidates, totalling 350 fits


[Parallel(n_jobs=4)]: Done 265 tasks      | elapsed:    1.6s


---------------------------------------------------------------------------------------------------------------------------------------

Logistic Regression (SMOTE) results:

accuracy: 0.8583399092861603
precision: 0.1659999358261316
recall: 0.8456666666666667
f1: 0.2774776959548837
Best Estimator = {} LogisticRegression(C=1000, class_weight='balanced', solver='newton-cg')

---------------------------------------------------------------------------------------------------------------------------------------


[Parallel(n_jobs=4)]: Done 350 out of 350 | elapsed:    2.7s finished


In [39]:
# Printing the classification report

label = ['Fin.Stable', 'Fin.Unstable']
pred_reg_sm = best_est_reg.predict(X_val_sm)
print(classification_report(y_val_sm, pred_reg_sm, target_names=label))

              precision    recall  f1-score   support

  Fin.Stable       1.00      0.87      0.93       923
Fin.Unstable       0.21      0.97      0.34        31

    accuracy                           0.88       954
   macro avg       0.60      0.92      0.64       954
weighted avg       0.97      0.88      0.91       954



In [41]:
CM = pd.DataFrame(confusion_matrix(y_val_sm, pred_reg_sm), columns = ['Survive','Bankrupt'], index = ['Survive','Bankrupt'])
CM

,Survive,Bankrupt
Survive,807,116
Bankrupt,1,30


# <center> Random Forest Classifier

In [42]:
accuracy_lst_rfc = []
precision_lst_rfc = []
recall_lst_rfc = []
f1_lst_rfc = []
auc_lst_rfc = []

rfc_sm = RandomForestClassifier()
#rfc_params = {}
rfc_params = {'max_features' : ['auto', 'sqrt', 'log2'],
              'random_state' : [42],
              'class_weight' : ['balanced','balanced_subsample'],
              'criterion' : ['gini', 'entropy'],
              'bootstrap' : [True,False]}
    
    
rand_rfc = GridSearchCV(rfc_sm,param_grid = rfc_params, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

for train, val in skfold.split(X_train_sm, y_train_sm):
    pipeline_rfc = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_rfc) # SMOTE happens during Cross Validation not before..
    model_rfc = pipeline_rfc.fit(X_train_sm, y_train_sm)
    best_est_rfc = rand_rfc.best_estimator_
    prediction_rfc = best_est_rfc.predict(X_train_sm[val])
    
    accuracy_lst_rfc.append(pipeline_rfc.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_rfc.append(precision_score(y_train_sm[val], prediction_rfc))
    recall_lst_rfc.append(recall_score(y_train_sm[val], prediction_rfc))
    f1_lst_rfc.append(f1_score(y_train_sm[val], prediction_rfc))
    auc_lst_rfc.append(roc_auc_score(y_train_sm[val], prediction_rfc))
    
print('---' * 45)
print('')
print("accuracy: {}".format(np.mean(accuracy_lst_rfc)))
print("precision: {}".format(np.mean(precision_lst_rfc)))
print("recall: {}".format(np.mean(recall_lst_rfc)))
print("f1: {}".format(np.mean(f1_lst_rfc)))
print('Best Estimator = {}',best_est_rfc)
print('---' * 45)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   15.9s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:   57.7s finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   14.6s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:   55.7s finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


...

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   14.3s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:   55.5s finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   15.2s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:   58.9s finished


---------------------------------------------------------------------------------------------------------------------------------------

accuracy: 1.0
precision: 1.0
recall: 1.0
f1: 1.0
Best Estimator = {} RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='entropy', random_state=42)
---------------------------------------------------------------------------------------------------------------------------------------


In [43]:
# Printing the classification report

label = ['Fin.Stable', 'Fin.Unstable']
pred_reg_rfc = best_est_rfc.predict(X_val_sm)
print(classification_report(y_val_sm, pred_reg_rfc, target_names=label))

              precision    recall  f1-score   support

  Fin.Stable       0.99      0.96      0.97       923
Fin.Unstable       0.34      0.68      0.46        31

    accuracy                           0.95       954
   macro avg       0.67      0.82      0.71       954
weighted avg       0.97      0.95      0.96       954



In [44]:
CM = pd.DataFrame(confusion_matrix(y_val_sm, pred_reg_rfc), columns = ['Survive','Bankrupt'], index = ['Survive','Bankrupt'])
CM

,Survive,Bankrupt
Survive,883,40
Bankrupt,10,21


In [45]:
### Now its time to saving the model

In [46]:
gsxgb.best_estimator_.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.06, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0599999987, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=120, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [47]:
file_name = 'Bankruptcy.sav'

pickle.dump(gsxgb.best_estimator_, open(file_name,'wb'))

In [48]:
loaded_model = pickle.load(open(file_name,'rb'))
loaded_model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [49]:
df_model

,Bankrupt?,ROA(A)_before_interest_and_%_after_tax,Persistent_EPS_in_the_Last_Four_Seasons,Per_Share_Net_profit_before_tax_(Yuan_¥),Debt_ratio_%,Net_worth/Assets,Borrowing_dependency,Net_profit_before_tax/Paid-in_capital,Net_Income_to_Total_Assets,Net_Income_to_Stockholder's_Equity
0,1,0.42,0.17,0.14,0.21,0.79,0.39,0.14,0.72,0.83
1,1,0.54,0.21,0.17,0.17,0.83,0.38,0.17,0.80,0.84
2,1,0.50,0.18,0.14,0.21,0.79,0.38,0.15,0.77,0.84
3,1,0.45,0.19,0.15,0.15,0.85,0.38,0.15,0.74,0.83
4,1,0.54,0.21,0.17,0.11,0.89,0.38,0.17,0.80,0.84
...,...,...,...,...,...,...,...,...,...,...
6814,0,0.54,0.22,0.17,0.12,0.88,0.37,0.17,0.80,0.84
6815,0,0.54,0.22,0.17,0.10,0.90,0.37,0.17,0.80,0.84
6816,0,0.53,0.21,0.17,0.04,0.96,0.37,0.17,0.80,0.84
6817,0,0.56,0.23,0.19,0.09,0.91,0.37,0.18,0.81,0.84


In [52]:
df_model_pred = pd.DataFrame({
    'ROA(A)_before_interest_and_%_after_tax': [0.56],
    'Persistent_EPS_in_the_Last_Four_Seasons': [0.18],
    'Per_Share_Net_profit_before_tax_(Yuan_¥)': [0.17],
    'Debt_ratio_%': [0.21],
    'Net_worth/Assets': [0.91],
    'Borrowing_dependency': [0.38],
    'Net_profit_before_tax/Paid-in_capital': [0.17],
    'Net_Income_to_Total_Assets': [0.77],
    "Net_Income_to_Stockholder's_Equity": [0.84]
})

In [53]:
df_model_pred

,ROA(A)_before_interest_and_%_after_tax,Persistent_EPS_in_the_Last_Four_Seasons,Per_Share_Net_profit_before_tax_(Yuan_¥),Debt_ratio_%,Net_worth/Assets,Borrowing_dependency,Net_profit_before_tax/Paid-in_capital,Net_Income_to_Total_Assets,Net_Income_to_Stockholder's_Equity
0,0.56,0.18,0.17,0.21,0.91,0.38,0.17,0.77,0.84


In [54]:
loaded_model.predict(df_model_pred)

array([1], dtype=int64)

In [56]:
loaded_model.predict_proba(df_model_pred)

array([[0.4868726, 0.5131274]], dtype=float32)